In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv


In [2]:
df=pd.read_csv("/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")
df.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
def classes(category):
        if(category=='ham'):
            return 1
        else:
           return 0

In [4]:
df['Category']=df['Category'].apply(classes)

In [5]:
df.tail(10)

,Category,Message
5562,1,Ok lor... Sony ericsson salesman... I ask shuh...
5563,1,Ard 6 like dat lor.
5564,1,Why don't you wait 'til at least wednesday to ...
5565,1,Huh y lei...
5566,0,REMINDER FROM O2: To get 2.50 pounds free call...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...
5571,1,Rofl. Its true to its name


In [6]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [7]:
df.describe()

,Category
count,5572.000000
mean,0.865937
std,0.340751
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [8]:
df.dtypes

Category     int64
Message     object
dtype: object

In [9]:
messages=np.asarray(df['Message'])

In [10]:
messages

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [11]:
classes=np.asarray(df['Category'])
classes

array([1, 1, 0, ..., 1, 1, 1])

In [12]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [13]:
final=[]

In [14]:
def total(messages):
    for i in messages:
        final.append(messages)  
    

In [15]:
df['Message'].nunique()

5157

In [16]:
tokenizer = Tokenizer(num_words=10000)#keep 10000 most frequent classes,ignore the others
tokenizer.fit_on_texts(messages)
sequences = tokenizer.texts_to_sequences(messages)

In [17]:
sequences[0]

[49,
 471,
 4426,
 840,
 755,
 658,
 65,
 8,
 1327,
 88,
 123,
 351,
 1328,
 148,
 2993,
 1329,
 67,
 58,
 4427,
 144]

In [18]:
len(sequences)

5572

In [19]:
len(sequences[0])

20

In [20]:
len(sequences[100])

11

In [21]:
max=0

In [22]:
for i in sequences:
    h=len(i)
    if(h>max):
        max=h

In [23]:
print('Maximum sequence length in the list of sequences:', max)

Maximum sequence length in the list of sequences: 189


# So this will be our maxlength of pad sequences

In [24]:
from keras.preprocessing.sequence import pad_sequences

In [25]:
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=189)

In [26]:
# word_index

In [27]:
data

array([[   0,    0,    0, ...,   58, 4427,  144],
       [   0,    0,    0, ...,  472,    6, 1940],
       [   0,    0,    0, ...,  660,  391, 2994],
       ...,
       [   0,    0,    0, ...,  107,  250, 9003],
       [   0,    0,    0, ...,  198,   12,   47],
       [   0,    0,    0, ...,    2,   61,  267]], dtype=int32)

In [28]:
X=data

In [29]:
X.shape

(5572, 189)

In [30]:
Y=df['Category']

In [31]:
import keras.utils

In [32]:
Y=keras.utils.to_categorical(Y)

In [33]:
Y

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [34]:
from sklearn.model_selection import train_test_split

# TRAIN TEST SPLIT

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5014, 189) (5014, 2)
(558, 189) (558, 2)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding,SimpleRNN,LSTM

In [40]:
model = Sequential()
model.add(Embedding(input_dim=10000,output_dim=32,input_length=189))
model.add(SimpleRNN(units=32))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
 metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 189, 32)           320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 322,146
Trainable params: 322,146
Non-trainable params: 0
_________________________________________________________________


In [41]:
batch_size = 60
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4011 samples, validate on 1003 samples
Epoch 1/10
4011/4011 [==============================] - 4s 995us/step - loss: 0.3375 - acc: 0.8901 - val_loss: 0.2012 - val_acc: 0.9467
Epoch 2/10
4011/4011 [==============================] - 4s 890us/step - loss: 0.1042 - acc: 0.9757 - val_loss: 0.0983 - val_acc: 0.9711
Epoch 3/10
4011/4011 [==============================] - 4s 881us/step - loss: 0.2001 - acc: 0.9187 - val_loss: 0.3307 - val_acc: 0.8475
Epoch 4/10
4011/4011 [==============================] - 4s 893us/step - loss: 0.0671 - acc: 0.9777 - val_loss: 0.0696 - val_acc: 0.9811
Epoch 5/10
4011/4011 [==============================] - 4s 890us/step - loss: 0.0333 - acc: 0.9905 - val_loss: 0.0714 - val_acc: 0.9806
Epoch 6/10
4011/4011 [==============================] - 4s 910us/step - loss: 0.0581 - acc: 0.9784 - val_loss: 0.2367 - val_acc: 0.8923
Epoch 7/10
4011/4011 [==============================] - 4s 896us/step - loss: 0.0407 - acc: 0.9889 - val_loss: 0.0851 - val_acc: 0.9786


In [42]:
acc = model.evaluate(X_test,Y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

558/558 [==============================] - 0s 361us/step
Test loss is 0.06 accuracy is 0.98 


# Test Accuracy turns to 98%.

In [45]:
def message_to_array(msg):
 msg = msg.lower().split(' ')
 test_seq = np.array([word_index[word] for word in msg])
 test_seq = np.pad(test_seq, (189-len(test_seq), 0),
 'constant', constant_values=(0))
 test_seq = test_seq.reshape(1, 189)
 return test_seq

In [47]:
custom_msg = 'Congratulations ur awarded 500 of CD vouchers or 125gift guaranteed and completely free entry for movies'
test_seq = message_to_array(custom_msg)
pred = model.predict_classes(test_seq)
print(pred)

[0]


# 0 means spam and the model also predicted correctly as spam..

In [50]:
custom_msg = 'Okay fine i will be coming with you we will be having a great time together'
test_seq = message_to_array(custom_msg)
pred = model.predict_classes(test_seq)
print(pred)

[1]


# 1 means ham and the model also predicted correctly as ham

# NOW TRYING FOR A LSTM MODEL

In [55]:
model1 = Sequential()
model1.add(Embedding(input_dim=10000,output_dim=32,input_length=189))
model1.add(LSTM(200, dropout_U=0.2,dropout_W=0.2))
model1.add(Dense(2,activation='sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 189, 32)           320000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               186400    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 402       
Total params: 506,802
Trainable params: 506,802
Non-trainable params: 0
_________________________________________________________________


In [58]:
batch_size = 60
model1.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4011 samples, validate on 1003 samples
Epoch 1/10
4011/4011 [==============================] - 31s 8ms/step - loss: 0.3261 - accuracy: 0.8894 - val_loss: 0.1196 - val_accuracy: 0.9696
Epoch 2/10
4011/4011 [==============================] - 32s 8ms/step - loss: 0.0617 - accuracy: 0.9825 - val_loss: 0.0743 - val_accuracy: 0.9801
Epoch 3/10
4011/4011 [==============================] - 30s 8ms/step - loss: 0.0287 - accuracy: 0.9915 - val_loss: 0.0644 - val_accuracy: 0.9806
Epoch 4/10
4011/4011 [==============================] - 31s 8ms/step - loss: 0.0184 - accuracy: 0.9960 - val_loss: 0.0653 - val_accuracy: 0.9831
Epoch 5/10
4011/4011 [==============================] - 30s 8ms/step - loss: 0.0118 - accuracy: 0.9974 - val_loss: 0.0769 - val_accuracy: 0.9791
Epoch 6/10
4011/4011 [==============================] - 31s 8ms/step - loss: 0.0062 - accuracy: 0.9980 - val_loss: 0.0695 - val_accuracy: 0.9801
Epoch 7/10
4011/4011 [==============================] - 30s 8ms/step - loss: 0.004

In [59]:
acc = model1.evaluate(X_test,Y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

558/558 [==============================] - 2s 3ms/step
Test loss is 0.06 accuracy is 0.99 


# Test Accuracy turns to 99% in case of LSTM model..

In [61]:
custom_msg = 'Congratulations ur awarded 500 of CD vouchers'
test_seq = message_to_array(custom_msg)
pred = model1.predict_classes(test_seq)
print(pred)

[0]


In [69]:
custom_msg1 = 'I got another job The one at the hospital doing data analysis or something starts on Monday Not sure when my thesis will finish'
test_seq = message_to_array(custom_msg1)
pred = model1.predict_classes(test_seq)
print(pred)

[1]


In [71]:
custom_msg1 = 'If he started searching he will get job in few days He has great potential and talent'
test_seq = message_to_array(custom_msg1)
pred = model1.predict_classes(test_seq)
print(pred)

[1]


# Our model is performing quite well..